In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

from proc_v2 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

2025-05-18 17:11:46.765759: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 17:11:46.911178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747555906.971806   39295 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747555906.989564   39295 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747555907.128671   39295 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
df_train['Duration'].nunique()

30

In [3]:
df_train['Heart_Rate'].nunique()

63

In [5]:
df_train['Body_Temp'].nunique()

75

In [12]:
df_train.groupby(['Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).sort_values('size', ascending = False)

mean       std  size
Duration Heart_Rate Body_Temp                          
9.0      91.0       39.700001  3.727759  0.123728  1085
24.0     104.0      40.700001  5.022945  0.109107   909
9.0      90.0       39.700001  3.724080  0.112540   860
         88.0       39.700001  3.663099  0.145242   810
21.0     99.0       40.700001  4.764327  0.088027   777
...                                 ...       ...   ...
16.0     80.0       40.599998  4.094345       NaN     1
                    40.900002  4.110874       NaN     1
         81.0       39.799999  3.761200       NaN     1
                    40.700001  4.521789       NaN     1
30.0     128.0      41.000000  5.690360       NaN     1

[12847 rows x 3 columns]

In [10]:
df_train[['Duration', 'Heart_Rate', 'Body_Temp']].drop_duplicates()

,Duration,Heart_Rate,Body_Temp
id,,,
0,26.0,101.0,41.000000
1,8.0,85.0,39.700001
2,7.0,84.0,39.799999
3,25.0,105.0,40.700001
4,25.0,102.0,40.599998
...,...,...,...
748567,18.0,94.0,40.490002
749505,9.0,70.0,40.200001
749553,1.0,91.0,38.500000
